# 준비작업

ToDo List
1. 사건번호로 채무자키 찾을 때 관할법원/관련사건법원 까지 함께 살피기(여러개 나오는 경우 공백으로 두기 때문에 수작업을 해야할 뿐 잘못된 키가 불려오지는 않을 것)
2. 1을 위해서 관할법원 정식명칭으로 맵핑하는 작업(일단은 지방> 지방법원 만 적용중. 현재 광주 아니면 부산이라 문제 없음)
3. 신규등록해야 하는 건 선별해서 법조치에, 기등록건은 법조치키를 법비용에
4. 대상자구분 확인하기
5. (전계좌, 부분계좌, 단일계좌 확인해야....)

### import

In [1]:
import pyautogui as au
import time
import pandas as pd
import re
from os.path import join
import os
import numpy as np
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException, NoSuchWindowException # NoSuchElementException, UnexpectedAlertPresentException, WebDriverException,
import random
from file_function import read_filelist
from datetime import date
from datetime import timedelta
import traceback
# 정렬순서 : 확정일자(내림), 법원(올림), 사건번호(내림)

### 함수, 전역변수

In [2]:
##################################
# 사건번호 눌러보고 집행문 발급 가능한 것만으로 목록작성 하도록!!!!!!!!!
lastday = "230810"
##################################
excel_dir = join(r"D:\4.관리\전자소송", lastday)
excel_file = "집행문신청_" + lastday
path_excel = join(excel_dir, excel_file+".xlsx")
bank = "신한은행"
driver, wait = None, None

# pause
def lwt() : time.sleep(2.5)
def mwt() : time.sleep(1)
def swt() : time.sleep(random.uniform(0.1,0.4))

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def con_scourt() :
    driver = set_chrome_driver()
    driver.get("https://ecfs.scourt.go.kr/ecf/index.jsp")
    wait = WebDriverWait(driver, 30) 
    wait.until(EC.element_to_be_clickable((By.ID, 'userId1')))
    mwt()
    return driver, wait

def certify() :
    elem = wait.until(EC.element_to_be_clickable((By.ID, "xwup_certselect_tek_input1")))
    mwt()
    elem.send_keys("help8700765!")
    swt()
    driver.find_element(By.ID, 'xwup_OkButton').click() # 인증서 확인
    mwt()

def position_new_win():
    position = driver.get_window_position()
    x = position.get('x')
    y = position.get('y')
    return x, y

def login() : 
    # 로그인(id) # 셀레니움 안 돼서.. 
    x, y = position_new_win()
    au.click(x+100,y+305)
    time.sleep(0.2)
    au.write("help")
    au.press("num2")
    au.press("num2")
    au.write("55")
    driver.find_element(By.ID, "userId1").send_keys(Keys.ENTER)
    # 로그인(인증서)
    certify()
    lwt()
############ 공통 사용 #############
def apply_or_print(AorP : int) :
    "apply_or_print : 신청 : 1, 발급 : 2"
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gnb"]/ul/li[4]/a'))).send_keys(Keys.ENTER) # 제증명
    driver.find_element(By.XPATH, '//*[@id="tmenu04"]/div/ul/li['+str(AorP)+']/a').send_keys(Keys.ENTER) # 제증명신청/발급
    swt()

def select_court_event(court, event) :
    """법원 및 사건번호 입력"""
    # 법원
    Select(driver.find_element(By.ID, 'cbxBubCd')).select_by_visible_text(court) # 집행문 선택
    swt()
    # 사건번호 선택
    event = re.search("(\d{4,4})([가-힣]+)(\d+)", event)
    year = event[1]
    sign = event[2]
    sn = event[3]
    Select(driver.find_element(By.XPATH, '//select[@title="사건연도"]')).select_by_value(year) # 연도 선택
    Select(driver.find_element(By.XPATH, '//select[@title="사건구분"]')).select_by_visible_text(sign)  # 사건구분 선택
    driver.find_element(By.XPATH, '//input[contains(@title,"사건일련번호")]').send_keys(sn)  # 사건일련번호 선택
    swt()
    # 출력할 때는 없는 버튼
    # driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
    # mwt()

########### 제증명 신청 관련 #################
def apply_certificates(docu_kind : int) :
    "docu_kind : 확정증명:0 | 송달증명:1 | 집행문:3"
    apply_or_print(1) # 1신청, 2발급
    Select(driver.find_element(By.ID, 'cbxDocCd')).select_by_index(docu_kind) #확정증명:0 | 송달증명:1 | 집행문:3
    swt()

def select_parties_new_win() : ##############
    # 양수금, 대여금, 신용카드이용대금 인지
    target_event_name = "양수금|대여금|신용카드이용대금"
    element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="saNm"]')))
    swt()
    eventname = element.text
    if re.search(target_event_name, element.text) == None : # 타겟 사건명이 아니면
        return False, eventname
    else :
        # 발급당사자 버튼
        wait.until(EC.element_to_be_clickable((By.ID, 'btnSelBalDs'))).click() # 발급당사자선택 버튼
        mwt()

        # 새창 전환
        driver.switch_to.window(driver.window_handles[1]) # 창 전환
        # 팝업창 컨트롤(발급당사자)
        driver.find_element(By.ID, 'allCheck').click() # 당사자 선택
        driver.find_element(By.ID, 'btnReg').click() # 등록
        swt()
        driver.find_element(By.ID, 'selectedAllCheck').click()  # 선택된 당사자 선택
        driver.find_element(By.ID, 'btnSetDs').click() # 확인
        swt()

        try : # 문제가 있다면 alert창이 뜬다. 위의 타겟 사건명도 모두 포함함
            Alert(driver).accept()
            driver.find_element(By.ID, 'btnClose').click() # 취소
            driver.switch_to.window(driver.window_handles[0]) # 창 전환
            return False, eventname
        except (NoSuchWindowException, NoAlertPresentException) : # 알람창 안 떴으니 이게 정상 상태
            # 창 되돌리기
            driver.switch_to.window(driver.window_handles[0]) # 창 전환
            return True, eventname

def check_document() : #############
    time.sleep(7) # pdf문서 대기
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="pdfReader"]/iframe')))
    
    # 문서 내용 확인 및 스크롤 (안 되네... )
    # driver.switch_to.frame(driver.find_elements(By.XPATH, '//*[@id="pdfReader"]/iframe')) # 문서viewer iframe 전환
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 문서 끝까지 스크롤 
    # driver.find_element(By.XPATH, '//*[@id="body"]/sd-root/sd-main/sd-toolbar/section[3]/section[2]/div/pu-button/div/pu-icon').click()
    
    # 문서 이상없음 체크 및 확인버튼
    driver.find_element(By.ID, 'guidcheck').click() # 문서이상없음
    driver.find_element(By.ID, 'btnNextPage').click() # 확인
    mwt()

def send_confirm() :
    # 제출 및 확인창
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendJchul'))).click() # 제출 버튼
    swt()
    Alert(driver).accept() # 확인alert
    mwt()
    
########## print ############
def confirm_print() :   # 게시물에 맞는 발급버튼 누르기
    while(True) :
        win_print = au.getWindowsWithTitle("인쇄")
        if win_print == [] :
            time.sleep(3)
        else :
            break
    win_print[0].activate()
    time.sleep(0.2)
    au.press('enter')

def print_execution() :
    "사건번호로 조회 후 집행문 발급"
    kind_docu = "집행문(정본포함)"
    try :
        driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a').click() # kind_docu의 발급 버튼 누르기
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tbMain1"]/tbody/tr/td[4]/a'))).click() # 출력버튼
    mwt()
    Alert(driver).accept() # 컨펌창 확인
    time.sleep(6) # 인쇄창 대기
    confirm_print() # 인쇄창 확인버튼 누르는 함수
    time.sleep(5) 
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnPrev"]'))).click() # 이전버튼 
    mwt()
    return "발급"

def print_fix_delivery(kind_docu:str) :
    "사건번호로 조회 후 확정증명, 송달증명 발급"
    # 확정증명 발급. a태그가 없는 경우 td의 text값을 리턴(기발급건은 '발 급'으로 정상적인 '발급'과 구별된다.)
    try :
        driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]/a').click() # 확정증명의 발급 버튼 누르기
    except :
        return driver.find_element(By.XPATH, '//a[@title="'+kind_docu+'"]/parent::td/following-sibling::td[4]').text
    mwt()
    Alert(driver).accept()
    time.sleep(6)
    # 인쇄 확인 버튼
    confirm_print()
    # 송달증명 발급
    time.sleep(5)
    Alert(driver).accept() # 컨펌창 확인
    mwt()
    return "발급"

############## 통합 함수 ##############
def fix_delivery(which:int, v) :
    """확정증명 or 송달증명 신청 종합 함수 \n
    which : 확정 - 0, 송달 - 1 \t v : df.iterrows()[1]"""
    apply_certificates(which) # 제증명신청, 제증명 종류
    select_court_event(v.법원, v.사건번호) # 법원 사건번호
    driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
    mwt()
    wait.until(EC.element_to_be_clickable((By.ID, "sinGbnCd2"))).click() # 당사자별
    swt()
    select_parties_new_win() # 발급당사자선택 ~ 새창 컨트롤
    driver.find_element(By.ID, "btnComplete").click() # 작성완료
    mwt()
    check_document() # 문서확인
    send_confirm() # 제출 및 comfirm
    certify() # 인증서 
    lwt() # 제출 후 인증은 lwt()추가!
    wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
    swt()
    # df수정
    if which == 0 : v.확정 = "신청"
    elif which == 1 : v.송달 = "신청"

### 엑셀 열기 - 키 연결해서 채무상태 확인 후 진행

In [3]:
### 원본이 필요할 땐 원본을 다른 이름으로 복사해둬!
df_origin = pd.read_excel(path_excel, sheet_name="집행문").fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
df 

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,가상계좌번호,채무자키,계좌키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,1,광주지방법원,2023가소524835,민사25단독,2023.08.10,주식회사 솔림헬프자산관리대부,김연상,,,,,,,,,,,,
1,2,광주지방법원,2023가소527704,민사25단독,2023.08.09,주식회사 솔림헬프자산관리대부,김원휘,,,,,,,,,,,,
2,3,부산지방법원,2023가소543332,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,마준성,원고패,,,,,,,,,,,
3,4,부산지방법원,2023가소541725,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,이수애,원고패,,,,,,,,,,,
4,5,부산지방법원,2023가소540166,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,안병일,원고패,,,,,,,,,,,
5,6,부산지방법원,2023가소538484,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,안경모,원고패,,,,,,,,,,,
6,7,부산지방법원,2023가소538354,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,박정출,원고패,,,,,,,,,,,
7,8,부산지방법원,2023가소538316,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,조재만,원고패,,,,,,,,,,,
8,9,부산지방법원,2023가소536419,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,강수한,원고패,,,,,,,,,,,
9,10,부산지방법원,2023가소535362,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,박치애,원고패,,,,,,,,,,,


### df에 채무자키 계좌키 제소신청일 추가하기

In [4]:
# 법조치조회 업데이트필요하면 전산조회.ipynb에서 작업
# cols = 채무자키, 매각사구분, 법조치구분, 관할법원, 사건번호, 청구금액법원, 관련사건번호, 매각사
path_pkl = r'D:\1.전산,개발\3.파일서버관리\0.월별전산조회' 
##########################################
file_name = "법조치조회새창_20230621_0914스캔파일작업용"
###########################################
# pkl 읽기
df_law = pd.read_pickle(join(path_pkl, file_name + ".pkl"))
# 잘 읽어졌나 확인
df_law[df_law["사건번호"]=="2022가소582163"].계좌키.values[0][:9]

'200957567'

In [5]:
# 키매칭 후 집행문시트에 추가
for i, v in df.iterrows() :
    df_result = df_law[(df_law.관련사건번호== v.사건번호) |(df_law.사건번호== v.사건번호)].loc[:,["채무자키", "계좌키", "제소신청일"]].drop_duplicates(subset="채무자키").fillna("")
    
    df_result["채무자키"] = df_result["채무자키"].astype(int) # 일부러 int한듯
    if len(df_result) == 1 :
        df.loc[i, ["채무자키", "제소신청일"]] = np.concatenate(df_result[["채무자키","제소신청일"]].values).tolist() # 제소신청일과 접수일은 차이가 있을 수 있음.. .#######

        # df.loc[i, ["채무자키"]] = df_result["채무자키"].values
        df.loc[i, ["계좌키"]] = df_result["계좌키"].values[0][:9] # 다계좌키인경우 하나만 입력하기 위해 
        # df.loc[i, ["제소신청일"]] = df_result["제소신청일"].values
df.fillna("", inplace=True)
# 엑셀에 저장

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)
df[["사건번호", "채무자키","계좌키"]]

,사건번호,채무자키,계좌키
0,2023가소524835,20469235,200989358
1,2023가소527704,20500209,201027896
2,2023가소543332,20430711,200951515
3,2023가소541725,20440056,200961337
4,2023가소540166,20440339,200961651
5,2023가소538484,20432425,200953374
6,2023가소538354,20446504,200968242
7,2023가소538316,,
8,2023가소536419,20432327,200953269
9,2023가소535362,20431895,200952774


### 종료건 확인 후 엑셀 다시 열기
- 전산에서 최신 채무상태 및 담당자 확인

In [3]:
# 엑셀 수작업 후 다시 열기
df_origin = pd.read_excel(path_excel, sheet_name="집행문").fillna("")
df_origin.법원 = df_origin.법원.str.replace("지법", "지방법원")
df = df_origin.copy()
display(df.head())
print(len(df))

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,가상계좌번호,채무자키,계좌키,제소신청일,법조치등록,메모등록,법비용등록,판결문
0,1,광주지방법원,2023가소524835,민사25단독,2023.08.10,주식회사 솔림헬프자산관리대부,김연상,신청(납부),신청,신청,,56216352394999.0,20469235,200989358.0,2023-04-04,,,,
1,2,광주지방법원,2023가소527704,민사25단독,2023.08.09,주식회사 솔림헬프자산관리대부,김원휘,신청(납부),신청,신청,,56216352395084.0,20500209,201027896.0,2023-04-13,,,,
2,3,부산지방법원,2023가소543332,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,마준성,원고패,,,,,20430711,200951515.0,,,,,
3,4,부산지방법원,2023가소541725,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,이수애,원고패,,,,,20440056,200961337.0,,,,,
4,5,부산지방법원,2023가소540166,민사제27단독,2023.08.09,주식회사 솔림헬프자산관리대부,안병일,원고패,,,,,20440339,200961651.0,,,,,


45


### driver 열기

In [4]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

# 집행문 신청

In [5]:
# 대법원 조회
# 오류나면 무한대기 되어버리니 try-except하지말고 그냥 종료시키자.
cnt_success, cnt_fail = 0, 0
for i, v in df.iterrows() :
    if v.집행문 == "" :
        # 제증명 신청-집행문 : 3
        apply_certificates(3)
        # 법원 및 사건번호 및 확인
        select_court_event(v.법원, v.사건번호)
        driver.find_element(By.ID, 'btnConfirm').click() # 확인 버튼
        mwt()
        
        # 알람창이 뜨면 문제있는거(ex소취하) 따라서 continue
        try : # 알람창 뜸
            Alert(driver).text
            Alert(driver).dismiss()
            swt()
            df.loc[i, ["집행문","확정","송달"]] = "미대상(알람)"
            cnt_fail += 1
            continue
        except (NoSuchWindowException, NoAlertPresentException) : # 알람창 안 뜸 > 정상
            pass

        # 발급당사자
        if select_parties_new_win()[0] : pass # 양수금 류인경우
        else : 
            df.loc[i, ["집행문","확정","송달"]] = select_parties_new_win()[1] # 양수금이 아닌 경우
            cnt_fail += 1
            continue

        # 동의 및 작성완료
        driver.find_element(By.ID, 'chkAgrElcSo').click() # 동의 체크박스
        driver.find_element(By.ID, 'btnComplete').click() # 작성완료
        mwt()
        Alert(driver).accept() # confirm
        
        # 문서이상없음 및 확인
        check_document() 

        # 은행선택 및 납부
        swt()
        Select(driver.find_element(By.ID, 'vtBankCd')).select_by_visible_text(bank) # 은행 선택
        driver.find_element(By.ID, 'btnElcPay').click() # 납부 버튼
        swt()
        Alert(driver).accept() # confirm
        mwt()
        
        # 제출 및 confirm 확인
        send_confirm()
            
        # 인증서
        certify()
        lwt()

        # 계좌정보 담기 및 완료버튼
        account = driver.find_element(By.ID, 'acno').text # 계좌정보 담기
        wait.until(EC.element_to_be_clickable((By.ID, 'btnSendList'))).click() # 완료버튼
        swt()

        # df 수정 : 계좌번호 및 집행문 상태
        df.loc[i, "가상계좌번호"] = account
        df.loc[i, "집행문"] = "신청(미납부)"
        cnt_success += 1

# 드라이버 종료 및 엑셀 저장
print(f"성공 {cnt_success}건, 실패 {cnt_fail}건")
driver.quit()

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

성공 5건, 실패 0건


In [13]:
### finally 에러발생시 중간저장 
print(f"성공 {cnt_success}건, 실패 {cnt_fail}건")
driver.quit()

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

성공 6건, 실패 0건


# 집행문 신청 후(입금 전)

### 키연결 후 df_valid 설정

In [6]:
####################################################
cost_day = str(date.today()) # 경우에 따라서 직접 입력 (20230106)
df_valid = df[df.집행문=='신청(미납부)'].copy() # 집행문 발행할 목록
####################################################
cost = 450
len(df_valid)

12

## 이체~판결문목록까지 작성하기

In [7]:
# 1) 이체용 df만들기 : 집행문 신청 후 가상계좌번호 받고 난 뒤
dict_cols = {"이체" : ["입금은행", "입금계좌", "고객관리성명", "입금액", "출금통장표시내용", "입금통장표시내용", "입금인코드", "비고", "업체사용key"],
            "법조치" : ["법조치키", "채무자키","계좌키","대상자명","담당자","법조치구분","법조치세부","법조치상태","관할법원","사건번호","접수일","법조치메모"],
            "메모" : ["메모키","채무자키","주민번호","매각사","매각사구분","원채권사","메모내용","메모구분","등록일자","등록유저","공지여부","메모색상","약속일자","약속금액","약속구분","약속이행"],
            "법비용" : ["법비용키","채무자키","계좌키","법조치키","법비용일","법비용구분","법비용내역","법비용메모","기타비"],
            "판결문" : ["채무자키","채무자","법원","사건번호","재도_승계_등등"]
            }

content_display=[]
names = []
for i in df_valid.피고.str.replace("주식회사|유한회사|법인|대표|보증인?|\b[주유]\b|\(주|주\)|외\s?\d.*", "", regex=True) :
    
    # 법조치 대상자명용
    i = re.sub("[^가-힣a-zA-Z]", " ", i).strip()
    sign = re.search("\s", i)
    if sign :
        new_i = i[:sign.start()]
    else :
        new_i = i
    if len(new_i) >= 2 : 
        names.append(new_i)
    else : names.append(i)
    # 이체 화면표시용
    if len(new_i) > 3 : content = new_i[:3]+" 제증명"
    elif len(new_i) >= 2 :
        content = new_i + " 제증명"
    elif len(i) > 3 : 
        content = i[:3] + " 제증명"
    else :
        content = i + " 제증명"
    content_display.append(content)

df_transfer = pd.DataFrame(None, columns=dict_cols["이체"])
df_transfer["입금계좌"] = df_valid.가상계좌번호.astype(dtype="int64")
df_transfer["출금통장표시내용"] = content_display
# 단일 값은 뒤에(그래야 모두 채워지니)
df_transfer["입금은행"] = "088"
df_transfer["입금액"] = cost

# 2) 법조치 : df에 채무자키 등 추가 후
df_legalaction = pd.DataFrame(None, columns=dict_cols["법조치"])
df_legalaction["채무자키"] = df_valid.채무자키
df_legalaction["계좌키"] = df_valid.계좌키
df_legalaction["대상자명"] = names
df_legalaction["관할법원"] = df_valid.법원
df_legalaction["사건번호"] = df_valid.사건번호
df_legalaction["접수일"] = df_valid.제소신청일
df_legalaction["법조치구분"] = "본안소송"
df_legalaction["법조치세부"] = "전자소송"
df_legalaction["법조치상태"] = "법조치완료"

# 3) 메모
date.today()
df_memo = pd.DataFrame(None, columns=dict_cols["메모"])
df_memo["채무자키"] = df_valid.채무자키
df_memo["메모내용"] = df_valid.사건번호 + " 발급"
df_memo["등록일자"] = str(date.today() + timedelta(days=3)) # 금요일에 작업하고, 월요일에 등록하는 경우 
df_memo["메모구분"] = "기타"
df_memo["등록유저"] = "전병만"

# 4) 법비용 #법조치키는 전산에서 조회해서 수동update
######## 수정일, 수정유저(나) 검색 셀추가 및 필터링 ########
######## 검색 결과 정렬이 바뀌니, 엑셀파일과 검색결과의 정렬 맞춘 후 복붙 ########
df_cost = pd.DataFrame(None, columns=dict_cols["법비용"])
df_cost["채무자키"] = df_valid.채무자키
df_cost["계좌키"] = df_valid.계좌키
df_cost["법비용일"] = cost_day
df_cost["법비용구분"] = "기본비용"
df_cost["법비용내역"] = "초도부여-신청비용" # 집행문부여
df_cost["집행문발급비"] = cost

# 5) 판결문
df_list = pd.DataFrame(None, columns=dict_cols["판결문"])
df_list["채무자키"] = df_valid.채무자키
df_list["채무자"] = df_valid.피고
df_list["법원"] = df_valid.법원
df_list["사건번호"] = df_valid.사건번호

#### 엑셀 작성

In [8]:
dict_sheet_df = {"이체": df_transfer, "법조치":df_legalaction,"메모":df_memo,"법비용":df_cost,"판결문":df_list}

with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

# 확정증명원/송달증명원 신청

In [10]:
driver, wait = con_scourt()
login() # 로그인 : 수동으로 하자. 인증서 되는 거 확인

In [11]:
cnt_success_fix, cnt_success_del, cnt_fail = 0, 0, 0
for i, v in df.iterrows() :
    if v.집행문.find('신청') >= 0 :
        try :
            if v.확정 == "" :
                fix_delivery(0, v)
                df.loc[i, "확정"] = "신청" # df수정
                cnt_success_fix += 1

            if v.송달 == "" :
                fix_delivery(1, v)
                df.loc[i, "송달"] = "신청" # df수정
                cnt_success_del += 1
        except :
            cnt_fail += 1

print(f"신청성공[확정증명원 : {cnt_success_fix}건, 송달증명원 : {cnt_success_del}건],    실패 {cnt_fail}건")
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

신청성공[확정증명원 : 12건, 송달증명원 : 12건],    실패 0건


# 출력

In [5]:
cnt_success, cnt_success_fix, cnt_success_del, cnt_fail = 0, 0, 0, 0
for i, v in df.iterrows() :
    ###################
    #if v.번호 < 2 :
    ###################
    try :
        # 1) 로그인 후 발급할 것이 있는 사건번호를 검색(조회)
        if (re.search("신청",v.집행문)) or (v.확정 == "신청") or (v.송달 == "신청") : 
            apply_or_print(2) # 제증명 발급
            driver.find_element(By.ID, "selSa").click() # 사건번호 radio
            swt()
            select_court_event(v.법원, v.사건번호)
            driver.find_element(By.ID, "btnRetrieve").click()
            mwt()
        
        # 2) 해당하는 것을 모두 출력(발급)
        if re.search("신청",v.집행문) : # v.집행문은 str임... 미납부 > 납부 수정 깜박하는 경우가 많아서.. '==신청(납부)'에서 바꿈
            exe_result = print_execution() # 집행문(정본포함) 하나이므로 매개변수 없이 지역변수로 선언함
            df.loc[i, "집행문"] = exe_result
            if exe_result == "발급" : cnt_success += 1

        if v.확정 == "신청" :
            fix_result = print_fix_delivery("확정증명")
            df.loc[i, "확정"] = fix_result
            if fix_result == "발급" : cnt_success_fix += 1
            
        if v.송달 == "신청" : # 송달도 발급해야 하면 연속 발급
            del_result = print_fix_delivery("송달증명")
            df.loc[i, "송달"] = del_result
            if del_result == "발급" : cnt_success_del += 1
    except Exception as e :
        print(traceback.format_exc())
        cnt_fail += 1
        continue

print(f"출력성공[집행문:{cnt_success}건, 확정:{cnt_success_fix}건, 송달:{cnt_success_del}건],   실패 : {cnt_fail}건")
# 드라이버 종료 및 엑셀 저장
driver.quit()
# 엑셀 저장
with pd.ExcelWriter(path_excel, engine='openpyxl', mode = 'a', if_sheet_exists = 'replace') as writer:
    df.to_excel(writer, sheet_name = "집행문", index=False)

출력성공[집행문:12건, 확정:12건, 송달:12건],   실패 : 0건


# 발급 및 스캔 후 : 스캔파일

In [6]:
# 스캔파일 목록을 df_scan에 추가하기
##################################################
df_valid = df[df.집행문=='발급'].copy().reset_index(drop=True) # 실제 작업할 영역 불러오기
# df_valid = df.iloc[:10,:].copy() # 실제 작업할 영역 불러오기
##################################################
path = r"D:\scan"
file_list = read_filelist(path) # 탐색기 정렬과 무관하게 이름 오름차순으로 정렬된 리스트 반환
df_valid["file"] = file_list
df_valid

,번호,법원,사건번호,재판부,확정일자,원고,피고,집행문,확정,송달,비고,가상계좌번호,채무자키,계좌키,제소신청일,법조치등록,메모등록,법비용등록,판결문,file
0,1,광주지방법원,2023가소524835,민사25단독,2023.08.10,주식회사 솔림헬프자산관리대부,김연상,발급,발급,발급,,56216352394999.0,20469235,200989358.0,2023-04-04,,,,,20230814101435.pdf
1,2,광주지방법원,2023가소527704,민사25단독,2023.08.09,주식회사 솔림헬프자산관리대부,김원휘,발급,발급,발급,,56216352395084.0,20500209,201027896.0,2023-04-13,,,,,20230814101449.pdf
2,14,광주지방법원,2023가소535712,민사25단독,2023.08.08,주식회사 솔림헬프자산관리대부,박순근,발급,발급,발급,,56216352395169.0,20499072,201026759.0,2023-05-12,,,,,20230814101501.pdf
3,15,광주지방법원,2023가소535682,민사25단독,2023.08.08,주식회사 솔림헬프자산관리대부,정모은,발급,발급,발급,,56216352395243.0,20466108,200986158.0,2023-05-12,,,,,20230814101515.pdf
4,16,광주지방법원,2023가소529946,민사25단독,2023.08.08,주식회사 솔림헬프자산관리대부,이유진,발급,발급,발급,,56216352395347.0,20464801,200984851.0,2023-04-20,,,,,20230814101528.pdf
5,17,광주지방법원,2023가소529144,민사25단독,2023.08.08,주식회사 솔림헬프자산관리대부,김규령,발급,발급,발급,,56216352395420.0,20499520,201027207.0,2023-04-19,,,,,20230814101542.pdf
6,18,광주지방법원,2023가소509423,민사26단독,2023.08.08,주식회사 솔림헬프자산관리대부,박금자,발급,발급,발급,,56216352397408.0,20429697,200949807.0,2023-02-07,,,,,20230814101554.pdf
7,19,광주지방법원,2022가소564843,민사25단독,2023.08.08,주식회사 솔림헬프자산관리대부,김동수,발급,발급,발급,,56216352397959.0,20428965,200948737.0,2022-09-19,,,,,20230814101608.pdf
8,24,부산지방법원,2023가소531476,민사제24단독,2023.08.04,주식회사 솔림헬프자산관리대부,오영민,발급,발급,발급,,56216352398082.0,20421513,200938792.0,,,,,,20230814101620.pdf
9,28,광주지방법원,2023가단524374,민사 1단독,2023.08.02,주식회사 솔림헬프자산관리대부,농업회사법인 올림 주식회사 외 1명,발급,발급,발급,,56216352398114.0,20423461,200941602.0,2023-05-12,,,,,20230814101632.pdf


In [7]:
# df_valid 한행씩 불러와서 파일명 바꾸기
success, fail = [], []
for i, v in df_valid.iterrows() :
    
    ext = os.path.splitext(v.file)[1]
    new_name = "_".join([str(int(v.채무자키)), v.피고, v.사건번호]) + ext
    try :
        os.rename(join(path, v.file), join(path, new_name))
        success.append([path, v.file, new_name])
    except Exception as e :
        fail.append([path, v.file, new_name, e.__class__])
        continue
print(f"총 {len(df_valid)}개 중, {len(success)}개 파일명 변경, {len(fail)}개 파일명 변경 실패")

총 12개 중, 12개 파일명 변경, 0개 파일명 변경 실패


In [ ]:
# 옛날꺼
# path = r"D:\scan\새 폴더"
# p_e = re.compile("\d{4}[가-힣]{1,3}\d+")
# df_result = pd.DataFrame(columns=["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"])
# for f in os.listdir(path) :
#     stem, ext = os.path.splitext(f)
#     e_name = p_e.search(stem).group()
#     # drop_duplicates(subset=["채무자키"], keep='first').
#     keys = df_law[(df_law.관련사건번호== e_name) |(df_law.사건번호== e_name)].loc[:,["채무자키", "계좌키", "사건번호", "계좌키", "제소신청일"]]
#     df_result = pd.concat([df_result, keys])
    
# df_result    

#### 법조치 등록 후 법비용에 법조치키 수동 입력
- 법조치새창 > 사건번호 멀티검색 > "법조치시트"에서 사건번호 가져와 > 수정유저 컬럼 추가 후 '자신'으로 필터링
- 순서 다르므로 엑셀과 전산프로그램 둘다 채무자키로 정렬 후 "법비용시트"에 복붙